## Tablas 

- dt_fi_tipodocumento
- dt_fi_tipomateria
- dt_fi_tiponacionalidades
- dt_fi_tipoorigenact
- dt_fi_tipoterminofiscalizacion
- dt_fi_unidadorigen

In [ ]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np

import pipeline.sql as plsql
import pipeline.metrics as plmet
import pipeline.eda as pleda
import functions_eda as funeda

In [ ]:
engine = plsql.create_engine('../config.yaml')
role = 'direccion_trabajo_inspections_write'

In [ ]:
def first_row_print(engine, role, schema, table, nrow = 5):
    qry = """set role {};
    select * 
    from {}.{}
    limit {}""".format(role, schema, table, nrow)
    
    return plsql.query(qry, engine)

In [ ]:
def tables_extraction(engine, role, table_schema):
    qry = """set role {};
        SELECT table_name
        FROM information_schema.Tables
        WHERE table_schema = '{}'""".format(role, table_schema)
    tab_names = plsql.query(qry, engine)
    columns = tab_names.table_name[[str.startswith(x, "dt_") for x in tab_names.table_name]]
    return columns

In [ ]:
def schema_column_info(engine, role, table_schema, table_name):
    qry = """set role {};
            SELECT *
            FROM information_schema.columns
            WHERE table_schema = '{}'
              AND table_name   = '{}'""".format(role, table_schema, table_name)
    
    tab = plsql.query(qry, engine)
    
    return tab

In [ ]:
def table_name_column_exists(role, engine, table_schema, table_name, column_name):
    qry = """set role {};
            SELECT *
            FROM information_schema.columns
            WHERE table_schema = '{}'
              AND table_name   = '{}'""".format(role, table_schema, table_name)
    columns = plsql.query(qry, engine).column_name


    bins = (column_name == columns)
    if (sum(bins) > 0):
        return table_name

In [ ]:
def describe_table(engine, role, schema, table, nrow = 5):
    
    print("\n------\n")
    print("Table: "  + str(table) )
    nrows = pleda.total_rows(engine, role, schema, table)
    print(" number of rows: " + str(nrows.iloc[0]))

    tab_head = first_row_print(engine, role, schema, table, nrow)
    print(" columns: " + str(tab_head.columns))
    
    print("\n")
    print(tab_head)
    
    tables_list = tables_extraction(engine, role, schema)
    tables_list = tables_list[(table == tables_list) == False]
    
    
    print("\n")
    print("Column description:\n")
    
    for col in tab_head.columns:
        prop_nulls = pleda.proportion_nulls(engine, role, schema, table, col)
        print ('. . . . .\n' + col + ':')
        
        list_columns = list()
        for tab in tables_list:
            ll = table_name_column_exists(role, engine, table_schema = 'public', 
                             table_name = tab, 
                             column_name = col)
            if (pd.isnull(ll) == False):
                list_columns.append(ll)

        print('proportion of nulls: ' + str(round(100*prop_nulls)) + '%' )
        print("# of tables same column: " + str(len(list_columns)) )
        print("% of tables same column: " + 
                  str(round(100*len(list_columns)/len(tables_list))) + "%")
        #print(list_columns)

    print("\n")
    print("\n")
    

In [ ]:
tablas_l = ['dt_fi_tipodocumento', 'dt_fi_tipomateria', 'dt_fi_tiponacionalidades', 
            'dt_fi_tipoorigenact', 'dt_fi_tipoterminofiscalizacion', 'dt_fi_unidadorigen']

In [ ]:
tablas_l = tables_extraction(engine, role, 'public')

In [ ]:
for k in tablas_l:
    describe_table(engine, role, schema = 'public', table = k, nrow = 5)

In [ ]:
list_tables_comp = tables_extraction(engine, role, 'public')
for tab in list_tables_comp:
    print("-----------\nTabla: " + tab)
    print(schema_column_info(engine, role, 'public', tab).column_name)
    print(first_row_print(engine, role, 'public', tab) )
    
    print("fin")